# **2022년도 따릉이 대여 데이터 전처리**
### 1. 따릉이 대여 데이터셋 중 이용거리 데이터를 확인해 본 결과, 0m와 같이 이용했다고 보기 어려운 이상치들이 존재함
### 2. 따라서 타당한 근거를 바탕으로 이용거리의 기준이 필요함
### 3. 근거로 서울시의 위치한 따릉이 대여소 간의 평균 거리를 사용하기로 함. 왜냐하면 따릉이는 대여소 대여, 대여소 반납이 원칙이므로 최소한 대여 대여소부터 반납 대여소까지의 이동은 필수적임
### 4. 그 결과, 평균적으로 2.08446951574621km 임을 확인
### 5. 월별로 제공하는 따릉이 대여이력 데이터들의 컬럼명 중 이용거리(M)으로 일괄 통일하고 하나의 파일로 합침
### 6. 프로젝트 진행에 있어 불필요한 컬럼은 삭제
### 7. 자치구별 분석이 필요하므로 따릉이 대여소 고유번호를 바탕으로 자치구 컬럼 생성
### 8. 담당하는 자치구만 추출함. 중요한 것은 대여지점과 반납지점이 모두 동일한 지역구인 경우만 추출하는 것!



In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import font_manager, rc

## 1. 대여소의 평균 거리 계산

In [ ]:
# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/따릉이위치정보.csv', encoding='utf-8')

# 자치구별 대여소 평균 거리 계산(degree -> km)
df_gu = df.groupby('자치구').apply(lambda x: 
                (((x['위도'].diff() * 111.139)**2 + (x['경도'].diff() * 111.139)**2)**0.5).mean(skipna=True))
df_gu = df_gu.reset_index(name='평균거리(km)')

# 결과 확인
print(df_gu)

# 자치구의 평균거리의 평균 계산
gu_mean = df_gu.groupby('자치구')['평균거리(km)'].mean()
mean_distance = gu_mean.mean()
print('총 평균 거리=',mean_distance)

# 시각화
plt.rc('font', family='NanumBarunGothic') 
sorted_df_gu = df_gu.sort_values('평균거리(km)')
plt.figure(figsize=(14, 10))
plt.bar(sorted_df_gu['자치구'], sorted_df_gu['평균거리(km)'])
plt.title('Average distance between bike rental stations by district in Seoul')
plt.xlabel('gu')
plt.ylabel('mean distance(km)')
plt.xticks(rotation=45)

# 평균거리의 평균값을 빨간색 점선으로 추가
mean_distance = gu_mean.mean()
plt.axhline(mean_distance, color='r', linestyle='--', label='평균거리의 평균값')

# 범례 추가
plt.legend()

plt.show()

plt.savefig('mean distance.png')

## 2. 컬럼 통일

In [ ]:
import pandas as pd

# csv 파일 읽어오기
df = pd.read_csv('/content/drive/MyDrive/서울특별시 공공자전거 대여이력 정보_2212.csv', encoding='cp949')

# 컬럼명 변경
df = df.rename(columns={'이용거리': '이용거리(M)'})
df.columns

## 3. 이용거리 기준 이상인 지점 선정하여 이상치 제거


In [ ]:
# 이용거리(M) column이 대여소 간 평균거리 이상인 row 추출하기
df_over_dec = df[df['이용거리(M)'] >= 2084.46951574621]

# 추출된 결과 출력하기
print(df_over_dec)

# 새로운 csv 파일로 추출된 결과 저장하기
df_over_dec.to_csv('따릉이 대여이력_2022.12.csv', index=False, encoding='cp949')

In [ ]:
# 정제한 대여이력 데이터를 이동거리(M) 컬럼을 기준으로 합치기
import glob

# 모든 csv 파일을 읽어서 리스트에 저장
all_files = glob.glob('/content/drive/MyDrive/따릉이 대여이력/*.csv')

# 모든 csv 파일을 읽어서 데이터프레임으로 변환한 후 리스트에 저장
dfs = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, encoding='cp949')
    dfs.append(df)

merged_df = pd.concat(dfs, axis=0,ignore_index=True, sort=False)

# 불필요한 컬럼 삭제하기
merged_df = merged_df.drop(['자전거번호','대여거치대','이용시간','반납거치대','생년','성별','이용자종류','이용시간', '이용거리'], axis=1)

In [ ]:
merged_df.columns

In [ ]:
merged_df.head(20)

## 4. 자치구 컬럼 생성

In [ ]:
import pandas as pd

# 대여소 정보 파일 불러오기
bike_info = pd.read_csv('/content/drive/MyDrive/대여소정보.csv', encoding='utf-8')

# 2022년 따릉이 대여 정보 파일 불러오기
bike_rental = pd.read_csv('/content/drive/MyDrive/merged_2022_따릉이대여이력.csv', encoding= 'cp949')

# '대여소 정보' 파일에서 대여소 번호와 자치구 컬럼만 추출
bike_info_sub = bike_info[['대여소번호', '자치구']]

# 2022년 따릉이 대여 정보 파일과 '대여소 정보' 파일 merge
rental_info = pd.merge(bike_rental, bike_info_sub, left_on='대여 대여소번호', right_on='대여소번호', how='left')
rental_info = pd.merge(rental_info, bike_info_sub, left_on='반납대여소번호', right_on='대여소번호', how='left')

# 불필요한 컬럼 제거
rental_info.drop(['대여소번호_x', '대여소번호_y'], axis=1, inplace=True)

# 컬럼 이름 변경
rental_info.rename(columns={'자치구_x': '대여 자치구', '자치구_y': '반납 자치구'}, inplace=True)

In [ ]:
# 합쳐진 결과를 csv 파일로 저장
rental_info.to_csv('merged_2022_따릉이대여이력_수정.csv', index=False, encoding='cp949')

In [ ]:
rental_info.head()

## 5. 서울시 전체 이용구간 상위 50개

In [ ]:
import pandas as pd
df_rental = pd.read_csv('/content/drive/MyDrive/merged_2022_따릉이대여이력_수정.csv', encoding='cp949')
# 대여소 정보 데이터를 읽어옵니다.
df_station = pd.read_csv('/content/drive/MyDrive/대여소정보.csv')


# 상위 50개 구간을 구합니다.
top_50 = df_rental.groupby(['대여 대여소번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(50, '빈도수')

# 대여소 번호와 대여소 이름, 위도, 경도 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()

# top_50 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_50_merged = pd.merge(top_50, df_station_name, left_on='대여 대여소번호', right_on='대여소번호', how='left')
top_50_merged = pd.merge(top_50_merged, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_50_merged = top_50_merged.drop(columns=['대여소번호_x', '대여소번호_y'])

# 빈도수를 기준으로 내림차순으로 정렬합니다.
top_50_merged = top_50_merged.sort_values(by=['빈도수'], ascending=False).reset_index(drop=True)

# 백분율 컬럼을 추가합니다.
total_count = top_50_merged['빈도수'].sum()
top_50_merged['백분율'] = top_50_merged['빈도수'] / total_count * 100

# 컬럼 순서를 변경합니다.
top_50_merged = top_50_merged[['대여 대여소번호', '보관소(대여소)명', '위도', '경도_x', '반납대여소번호', '보관소(대여소)명_y', '위도_y', '경도_y', '빈도수', '백분율']]


In [ ]:
# 컬럼명 변경합니다.
top_50_merged = top_50_merged.rename(columns={'대여 대여소번호': '대여소 번호', '보관소(대여소)명_x': '대여소명', '위도_x':'대여소 위도', '경도_x':'대여소 경도', '반납대여소번호':'반납소 번호', '보관소(대여소)명_y':'반납소명','위도_y':'반납소 위도', '경도_y':'반납소 경도'})


top_50_merged.to_csv('서울시_2022_따릉이상위50구간.csv', index=False, encoding='cp949')

In [ ]:
top_50_merged

## 6. 담당 자치구 추출

In [ ]:
# csv 파일을 pandas DataFrame으로 읽어오기
df = pd.read_csv('/content/drive/MyDrive/merged_2022_따릉이대여이력_수정.csv', encoding='cp949')

# 자치구 컬럼 중 강서구, 양천구, 구로구, 영등포구, 금천구에 해당하는 행만 추출
df_filtered = df[((df['대여 자치구'] == '강서구') & (df['반납 자치구'] == '강서구')) | 
                 ((df['대여 자치구'] == '양천구') & (df['반납 자치구'] == '양천구')) | 
                 ((df['대여 자치구'] == '구로구') & (df['반납 자치구'] == '구로구')) |
                 ((df['대여 자치구'] == '금천구') & (df['반납 자치구'] == '금천구')) |
                 ((df['대여 자치구'] == '영등포구') & (df['반납 자치구'] == '영등포구'))]

# 결과를 csv 파일로 저장
df_filtered.to_csv('강서양천구로금천영등포_2022_따릉이대여이력.csv', index=False, encoding='cp949')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,8,9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# 결과확인
df_filtered.head()

## 6. 자치구 별로 파일 생성

In [ ]:
df = pd.read_csv ('/content/drive/MyDrive/강서양천구로금천영등포_2022_따릉이대여이력.csv', encoding = 'cp949')
df_filtered = df[((df['대여 자치구'] == '양천구') & (df['반납 자치구'] == '양천구'))]

df_station = pd.read_csv('/content/drive/MyDrive/대여소정보.csv')

df_merged = pd.merge(df_filtered, df_station[['대여소번호', '경도', '위도']], left_on='대여 대여소번호', right_on='대여소번호')
df_merged = pd.merge(df_merged, df_station[['대여소번호', '경도', '위도']], left_on='반납대여소번호', right_on='대여소번호', suffixes=('_대여', '_반납'))
df_merged = df_merged.rename(columns={'대여 대여소번호': '대여소 번호', '대여 대여소명': '대여소명', '위도_대여':'대여소 위도', '경도_대여':'대여소 경도', '반납대여소명':'반납소명','위도_반납':'반납소 위도', '경도_반납':'반납소 경도'})
df_merged.drop(['대여소번호_대여', '대여소번호_반납'], axis=1, inplace=True)

df_merged.to_csv('양천구_2022_따릉이대여이력.csv', index=False, encoding='cp949')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_merged.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
0,2022.12.1 0:26,4531,목동아파트 613동 앞,2022.12.1 0:48,722,경서농협 버스정류장(우리은행신정동지점방향),22.0,3110.00,ST-2844,ST-993,양천구,양천구,126.884087,37.533157,126.857384,37.521511
1,2022-01-01 11:37:34,4531,목동아파트 613동 앞,2022-01-01 12:02:46,722,경서농협 버스정류장(우리은행신정동지점방향),NaN,4480.00,NaN,NaN,양천구,양천구,126.884087,37.533157,126.857384,37.521511
2,2022-11-23 23:45:51,4531,목동아파트 613동 앞,2022-11-24 00:14:41,722,경서농협 버스정류장(우리은행신정동지점방향),28.0,2688.14,ST-2844,ST-993,양천구,양천구,126.884087,37.533157,126.857384,37.521511
3,2022-05-10 16:43:59,4531,목동아파트 613동 앞,2022-05-10 17:11:00,722,경서농협 버스정류장(우리은행신정동지점방향),NaN,3710.45,NaN,NaN,양천구,양천구,126.884087,37.533157,126.857384,37.521511
4,2022.12.1 17:35,764,목동청소년수련관,2022.12.1 18:01,722,경서농협 버스정류장(우리은행신정동지점방향),25.0,2390.00,ST-1505,ST-993,양천구,양천구,126.875893,37.531029,126.857384,37.521511


# **5개 구 최다 이용 대여소 추출 과정**

# *1. 구로구*

In [ ]:
# csv 파일을 pandas DataFrame으로 읽어오기
df_filtered = pd.read_csv('/content/구로구_2022_따릉이대여이력.csv', encoding='cp949')

# 자치구 컬럼이 '구로구'인 데이터만 추출
df_guro = df_filtered[df_filtered['대여 자치구'] == '구로구']

# '대여 대여소번호'와 '반납 대여소 번호' 컬럼의 조합 빈도수 계산
combo_counts = df_guro.groupby(['대여소 번호', '반납대여소번호']).size().reset_index(name='count')

# 가장 많이 나온 조합과 횟수 출력
max_combo = combo_counts.loc[combo_counts['count'].idxmax()]
print('가장 많이 이용한 구간:', max_combo['대여소 번호'], max_combo['반납대여소번호'])
print('2022년 이용 횟수:', max_combo['count'])
print('2022년 구로구 내 이동 전체 따릉이 이용 횟수:',  len(df_guro))

가장 많이 이용한 구간: 1911 1911
2022년 이용 횟수: 4042
2022년 구로구 내 이동 전체 따릉이 이용 횟수: 269034


In [ ]:
df_guro_max = df_filtered[(df_filtered['대여 자치구'] == '구로구') & (df_filtered['대여소 번호'] == 1911) & (df_filtered['반납대여소번호'] == 1911)]

# 결과 출력
df_guro_max.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
189004,2022.12.1 13:36,1911,구로디지털단지역 앞,2022.12.1 14:25,1911,구로디지털단지역 앞,48.0,8640.00,ST-668,ST-668,구로구,구로구,126.901321,37.48494,126.901321,37.48494
189005,2022.12.1 13:37,1911,구로디지털단지역 앞,2022.12.1 14:25,1911,구로디지털단지역 앞,47.0,9049.71,ST-668,ST-668,구로구,구로구,126.901321,37.48494,126.901321,37.48494
189006,2022.12.1 18:45,1911,구로디지털단지역 앞,2022.12.1 20:23,1911,구로디지털단지역 앞,98.0,4890.00,ST-668,ST-668,구로구,구로구,126.901321,37.48494,126.901321,37.48494
189007,2022.12.2 12:38,1911,구로디지털단지역 앞,2022.12.2 13:25,1911,구로디지털단지역 앞,46.0,8879.50,ST-668,ST-668,구로구,구로구,126.901321,37.48494,126.901321,37.48494
189008,2022.12.2 15:25,1911,구로디지털단지역 앞,2022.12.2 17:09,1911,구로디지털단지역 앞,103.0,8790.00,ST-668,ST-668,구로구,구로구,126.901321,37.48494,126.901321,37.48494


## 2. 영등포구

In [ ]:
# csv 파일을 pandas DataFrame으로 읽어오기
df_filtered = pd.read_csv('/content/영등포구_2022_따릉이대여이력.csv', encoding='cp949')

# 자치구 컬럼이 '구로구'인 데이터만 추출
df_yeong = df_filtered[df_filtered['대여 자치구'] == '영등포구']

# '대여 대여소번호'와 '반납 대여소 번호' 컬럼의 조합 빈도수 계산
combo_counts = df_yeong.groupby(['대여소 번호', '반납대여소번호']).size().reset_index(name='count')

# 가장 많이 나온 조합과 횟수 출력
max_combo = combo_counts.loc[combo_counts['count'].idxmax()]
print('가장 많이 이용한 구간:', max_combo['대여소 번호'], max_combo['반납대여소번호'])
print('2022년 이용 횟수:', max_combo['count'])
print('2022년 구로구 내 이동 전체 따릉이 이용 횟수:',  len(df_yeong))

가장 많이 이용한 구간: 207 207
2022년 이용 횟수: 19626
2022년 구로구 내 이동 전체 따릉이 이용 횟수: 607745


In [ ]:
df_yeong_1911 = df_filtered[(df_filtered['대여 자치구'] == '영등포구') & (df_filtered['대여소 번호'] == 207) & (df_filtered['반납대여소번호'] == 207)]

# 결과 출력
df_yeong_1911.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
255426,2022.12.1 16:25,207,여의나루역 1번출구 앞,2022.12.1 17:20,207,여의나루역 1번출구 앞,55.0,3270.00,ST-73,ST-73,영등포구,영등포구,126.9319,37.527157,126.9319,37.527157
255427,2022.12.1 16:27,207,여의나루역 1번출구 앞,2022.12.1 17:20,207,여의나루역 1번출구 앞,53.0,3000.87,ST-73,ST-73,영등포구,영등포구,126.9319,37.527157,126.9319,37.527157
255428,2022.12.1 16:30,207,여의나루역 1번출구 앞,2022.12.1 17:21,207,여의나루역 1번출구 앞,50.0,3091.98,ST-73,ST-73,영등포구,영등포구,126.9319,37.527157,126.9319,37.527157
255429,2022.12.1 16:32,207,여의나루역 1번출구 앞,2022.12.1 17:37,207,여의나루역 1번출구 앞,64.0,3342.83,ST-73,ST-73,영등포구,영등포구,126.9319,37.527157,126.9319,37.527157
255430,2022.12.1 16:32,207,여의나루역 1번출구 앞,2022.12.1 17:37,207,여의나루역 1번출구 앞,64.0,3120.00,ST-73,ST-73,영등포구,영등포구,126.9319,37.527157,126.9319,37.527157


## 3. 금천구

In [ ]:
# csv 파일을 pandas DataFrame으로 읽어오기
df_filtered = pd.read_csv('/content/금천구_2022_따릉이대여이력.csv', encoding='cp949')

# 자치구 컬럼이 '금천구'인 데이터만 추출
df_gh= df_filtered[df_filtered['대여 자치구'] == '금천구']

# '대여 대여소번호'와 '반납 대여소 번호' 컬럼의 조합 빈도수 계산
combo_counts = df_gh.groupby(['대여소 번호', '반납대여소번호']).size().reset_index(name='count')

# 가장 많이 나온 조합과 횟수 출력
max_combo = combo_counts.loc[combo_counts['count'].idxmax()]
print('가장 많이 이용한 구간:', max_combo['대여소 번호'], max_combo['반납대여소번호'])
print('2022년 이용 횟수:', max_combo['count'])
print('2022년 금천구 내 이동 전체 따릉이 이용 횟수:',  len(df_gh))

가장 많이 이용한 구간: 1827 1827
2022년 이용 횟수: 6710
2022년 금천구 내 이동 전체 따릉이 이용 횟수: 129332


In [ ]:
df_gh_1911 = df_filtered[(df_filtered['대여 자치구'] == '금천구') & (df_filtered['대여소 번호'] == 1827) & (df_filtered['반납대여소번호'] == 1827)]

# 결과 출력
df_gh_1911.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
28165,2022.12.1 20:08,1827,독산보도육교 앞 자전거 보관소,2022.12.1 20:29,1827,독산보도육교 앞 자전거 보관소,21.0,3990.0,ST-660,ST-660,금천구,금천구,126.893066,37.456726,126.893066,37.456726
28166,2022.12.2 12:52,1827,독산보도육교 앞 자전거 보관소,2022.12.2 14:30,1827,독산보도육교 앞 자전거 보관소,98.0,13300.0,ST-660,ST-660,금천구,금천구,126.893066,37.456726,126.893066,37.456726
28167,2022.12.2 15:02,1827,독산보도육교 앞 자전거 보관소,2022.12.2 16:47,1827,독산보도육교 앞 자전거 보관소,104.0,27280.0,ST-660,ST-660,금천구,금천구,126.893066,37.456726,126.893066,37.456726
28168,2022.12.2 16:29,1827,독산보도육교 앞 자전거 보관소,2022.12.2 17:22,1827,독산보도육교 앞 자전거 보관소,52.0,9990.0,ST-660,ST-660,금천구,금천구,126.893066,37.456726,126.893066,37.456726
28169,2022.12.2 22:25,1827,독산보도육교 앞 자전거 보관소,2022.12.2 23:02,1827,독산보도육교 앞 자전거 보관소,36.0,8080.0,ST-660,ST-660,금천구,금천구,126.893066,37.456726,126.893066,37.456726


## 4. 강서구

In [ ]:
# csv 파일을 pandas DataFrame으로 읽어오기
df_filtered = pd.read_csv('/content/강서구_2022_따릉이대여이력.csv', encoding='cp949')

# 자치구 컬럼이 '강서구'인 데이터만 추출
df_gs= df_filtered[df_filtered['대여 자치구'] == '강서구']

# '대여 대여소번호'와 '반납 대여소 번호' 컬럼의 조합 빈도수 계산
combo_counts = df_gs.groupby(['대여소 번호', '반납대여소번호']).size().reset_index(name='count')

# 가장 많이 나온 조합과 횟수 출력
max_combo = combo_counts.loc[combo_counts['count'].idxmax()]
print('가장 많이 이용한 구간:', max_combo['대여소 번호'], max_combo['반납대여소번호'])
print('2022년 이용 횟수:', max_combo['count'])
print('2022년 강서구 내 이동 전체 따릉이 이용 횟수:',  len(df_gs))

가장 많이 이용한 구간: 3789 3789
2022년 이용 횟수: 3655
2022년 강서구 내 이동 전체 따릉이 이용 횟수: 890418


In [ ]:
df_gs_1911 = df_filtered[(df_filtered['대여 자치구'] == '강서구') & (df_filtered['대여소 번호'] == 3789) & (df_filtered['반납대여소번호'] == 3789)]

# 결과 출력
df_gs_1911.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
236651,2022-04-01 10:59:22,3789,염창나들목,2022-04-01 12:12:02,3789,염창나들목,NaN,NaN,NaN,NaN,강서구,강서구,126.871559,37.556805,126.871559,37.556805
236652,2022-04-01 11:45:10,3789,염창나들목,2022-04-01 12:49:25,3789,염창나들목,NaN,NaN,NaN,NaN,강서구,강서구,126.871559,37.556805,126.871559,37.556805
236653,2022-04-01 11:44:41,3789,염창나들목,2022-04-01 12:49:38,3789,염창나들목,NaN,NaN,NaN,NaN,강서구,강서구,126.871559,37.556805,126.871559,37.556805
236654,2022-04-01 13:27:26,3789,염창나들목,2022-04-01 14:58:00,3789,염창나들목,NaN,NaN,NaN,NaN,강서구,강서구,126.871559,37.556805,126.871559,37.556805
236655,2022-04-01 18:48:14,3789,염창나들목,2022-04-01 19:18:41,3789,염창나들목,NaN,NaN,NaN,NaN,강서구,강서구,126.871559,37.556805,126.871559,37.556805


## 5. 양천구

In [ ]:
# csv 파일을 pandas DataFrame으로 읽어오기
df_filtered = pd.read_csv('/content/양천구_2022_따릉이대여이력.csv', encoding='cp949')

# 자치구 컬럼이 '강서구'인 데이터만 추출
df_gs= df_filtered[df_filtered['대여 자치구'] == '양천구']

# '대여 대여소번호'와 '반납 대여소 번호' 컬럼의 조합 빈도수 계산
combo_counts = df_gs.groupby(['대여소 번호', '반납대여소번호']).size().reset_index(name='count')

# 가장 많이 나온 조합과 횟수 출력
max_combo = combo_counts.loc[combo_counts['count'].idxmax()]
print('가장 많이 이용한 구간:', max_combo['대여소 번호'], max_combo['반납대여소번호'])
print('2022년 이용 횟수:', max_combo['count'])
print('2022년 양천구 내 이동 전체 따릉이 이용 횟수:',  len(df_gs))

가장 많이 이용한 구간: 780 780
2022년 이용 횟수: 4614
2022년 양천구 내 이동 전체 따릉이 이용 횟수: 401831


In [ ]:
df_yc_1911 = df_filtered[(df_filtered['대여 자치구'] == '양천구') & (df_filtered['대여소 번호'] == 780) & (df_filtered['반납대여소번호'] == 780)]

# 결과 출력
df_yc_1911.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
200697,2022.12.1 7:48,780,신정교 하부,2022.12.1 9:06,780,신정교 하부,77.0,19900.13,ST-2006,ST-2006,양천구,양천구,126.877647,37.517338,126.877647,37.517338
200698,2022.12.1 10:04,780,신정교 하부,2022.12.1 10:54,780,신정교 하부,50.0,7190.00,ST-2006,ST-2006,양천구,양천구,126.877647,37.517338,126.877647,37.517338
200699,2022.12.1 19:42,780,신정교 하부,2022.12.1 20:12,780,신정교 하부,30.0,5502.57,ST-2006,ST-2006,양천구,양천구,126.877647,37.517338,126.877647,37.517338
200700,2022.12.2 10:06,780,신정교 하부,2022.12.2 11:03,780,신정교 하부,56.0,7583.80,ST-2006,ST-2006,양천구,양천구,126.877647,37.517338,126.877647,37.517338
200701,2022.12.2 11:24,780,신정교 하부,2022.12.2 12:15,780,신정교 하부,51.0,10405.83,ST-2006,ST-2006,양천구,양천구,126.877647,37.517338,126.877647,37.517338


In [ ]:
df_top_total = pd.concat([df_guro, df_yc_1911, df_gs_1911, df_gh_1911, df_yeong_1911])
df_top_total
df_top_total.to_csv('구별_동일top_total.csv', index=False, encoding='cp949')

## 최다 경로를 해본 결과, 모두 같은 대여소에서 대여와 반납을 하는 것을 확인함. 이는 구간의 자전거 주행 환경을 확인하기 어려운 결과이므로 서로 다른 대여소 중 최다 이용 구간을 선정하는 것이 바람직할 것으로 사료됨. 아래는 관련 코드임


## 1. 양천구

In [ ]:
import pandas as pd
df = pd.read_csv ('/content/양천구_2022_따릉이대여이력.csv', encoding = 'cp949')

# 자치구 컬럼이 '양천구'이고 대여소번호와 반납 대여소번호가 서로 다른 경우 추출
df_yc = df.loc[(df['대여 자치구'] == '양천구') & (df['대여소 번호'] != df['반납대여소번호'])]

# 대여 대여소번호와 반납 대여소번호를 조합하여 새로운 컬럼 생성
df_yc['대여 반납'] = df_yc['대여소 번호'].astype(str) + '-' + df_yc['반납대여소번호'].astype(str)

# 가장 많이 나온 상위 3개의 조합과 그 횟수 추출
top_3 = df_yc['대여 반납'].value_counts().head(3)

len(df_yc)

<ipython-input-63-0fc633d81f63>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yc['대여 반납'] = df_yc['대여소 번호'].astype(str) + '-' + df_yc['반납대여소번호'].astype(str)


331499

In [ ]:
# 대여소 번호가 서로 다른 경우를 제외한 데이터만 추출
df_yc_unique = df_yc[df_yc['대여소 번호'] != df_yc['반납대여소번호']]

# 대여 대여소번호와 반납 대여소번호가 서로 다른 조합 상위 3개와 그 횟수 추출
top_3 = df_yc_unique.groupby(['대여 자치구', '대여소 번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(3, columns=['빈도수'])

In [ ]:
# 대여소 정보 데이터셋을 읽어옵니다.
df_station = pd.read_csv('/content/drive/MyDrive/대여소정보.csv', encoding = 'utf-8')

# 대여소 번호와 대여소 이름 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()

# top_3 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_3_merged = pd.merge(top_3, df_station_name, left_on='대여소 번호', right_on='대여소번호', how='left')
top_3_merged = pd.merge(top_3_merged, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')

# 빈도수의 백분율 계산
top_3_merged['백분율'] = (top_3_merged['빈도수'] / len(df_yc)) * 100

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_3_merged = top_3_merged.drop(columns=['대여소번호_x', '대여소번호_y'])
top_3_merged = top_3_merged.rename(columns={'보관소(대여소)명_x': '대여 대여소명', '보관소(대여소)명_y': '반납 대여소명','위도_x':'대여소 위도', '경도_x':'대여소 경도', '위도_y':'반납소 위도', '경도_y':'반납소 경도'})

top_3_merged

,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,양천구,703,734,1002,오목교역 7번출구 앞,37.523628,126.875244,신트리공원 입구,37.513950,126.856056,0.302263
1,양천구,770,787,928,목동역5번출구 교통정보센터 앞,37.525715,126.864868,양천아파트 입구,37.507904,126.862648,0.279941
2,양천구,787,792,843,양천아파트 입구,37.507904,126.862648,목동트라팰리스 웨스턴에비뉴,37.525372,126.870926,0.254299


## 2. 강서구

In [ ]:
df = pd.read_csv ('/content/강서구_2022_따릉이대여이력.csv', encoding = 'cp949')

# 자치구 컬럼이 '강서구'이고 대여소번호와 반납 대여소번호가 서로 다른 경우 추출
df_gs = df.loc[(df['대여 자치구'] == '강서구') & (df['대여소 번호'] != df['반납대여소번호'])]

# 대여 대여소번호와 반납 대여소번호를 조합하여 새로운 컬럼 생성
df_gs['대여 반납'] = df_gs['대여소 번호'].astype(str) + '-' + df_gs['반납대여소번호'].astype(str)

# 가장 많이 나온 상위 3개의 조합과 그 횟수 추출
top_3 = df_gs['대여 반납'].value_counts().head(3)

# 대여소 번호가 서로 다른 경우를 제외한 데이터만 추출
df_gs_unique = df_gs[df_gs['대여소 번호'] != df_gs['반납대여소번호']]

# 대여 대여소번호와 반납 대여소번호가 서로 다른 조합 상위 3개와 그 횟수 추출
top_3 = df_gs_unique.groupby(['대여 자치구', '대여소 번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(3, columns=['빈도수'])

# 대여소 번호와 대여소 이름 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()


# top_3 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_3_merged_gs = pd.merge(top_3, df_station_name, left_on='대여소 번호', right_on='대여소번호', how='left')
top_3_merged_gs = pd.merge(top_3_merged_gs, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')
# 빈도수의 백분율 계산
top_3_merged_gs['백분율'] = (top_3_merged_gs['빈도수'] / len(df_gs)) * 100

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_3_merged_gs = top_3_merged_gs.drop(columns=['대여소번호_x', '대여소번호_y'])
top_3_merged_gs = top_3_merged_gs.rename(columns={'보관소(대여소)명_x': '대여 대여소명', '보관소(대여소)명_y': '반납 대여소명','위도_x':'대여소 위도', '경도_x':'대여소 경도', '위도_y':'반납소 위도', '경도_y':'반납소 경도'})

top_3_merged_gs

<ipython-input-59-8ce1836ab44b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gs['대여 반납'] = df_gs['대여소 번호'].astype(str) + '-' + df_gs['반납대여소번호'].astype(str)


,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,강서구,1153,1150,1079,"발산역 1번, 9번 인근 대여소",37.558949,126.837174,송정역 1번출구,37.561531,126.810699,0.136903
1,강서구,2715,1125,946,마곡나루역 2번 출구,37.566925,126.827438,명덕고교입구(영종빌딩),37.552914,126.836571,0.120028
2,강서구,1125,2715,785,명덕고교입구(영종빌딩),37.552914,126.836571,마곡나루역 2번 출구,37.566925,126.827438,0.099601


## 3. 영등포구

In [ ]:
df = pd.read_csv ('/content/영등포구_2022_따릉이대여이력.csv', encoding = 'cp949')

# 자치구 컬럼이 '영등포구'이고 대여소번호와 반납 대여소번호가 서로 다른 경우 추출
df_yd = df.loc[(df['대여 자치구'] == '영등포구') & (df['대여소 번호'] != df['반납대여소번호'])]

# 대여 대여소번호와 반납 대여소번호를 조합하여 새로운 컬럼 생성
df_yd['대여 반납'] = df_yd['대여소 번호'].astype(str) + '-' + df_yd['반납대여소번호'].astype(str)

# 가장 많이 나온 상위 3개의 조합과 그 횟수 추출
top_3 = df_yd['대여 반납'].value_counts().head(3)

# 대여소 번호가 서로 다른 경우를 제외한 데이터만 추출
df_yd_unique = df_yd[df_yd['대여소 번호'] != df_yd['반납대여소번호']]

# 대여 대여소번호와 반납 대여소번호가 서로 다른 조합 상위 3개와 그 횟수 추출
top_3 = df_yd_unique.groupby(['대여 자치구', '대여소 번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(3, columns=['빈도수'])

# 대여소 번호와 대여소 이름 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()

# top_3 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_3_merged_yd = pd.merge(top_3, df_station_name, left_on='대여소 번호', right_on='대여소번호', how='left')
top_3_merged_yd = pd.merge(top_3_merged_yd, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')
# 빈도수의 백분율 계산
top_3_merged_yd['백분율'] = (top_3_merged_yd['빈도수'] / len(df_yd)) * 100

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_3_merged_yd = top_3_merged_yd.drop(columns=['대여소번호_x', '대여소번호_y'])
top_3_merged_yd = top_3_merged_yd.rename(columns={'보관소(대여소)명_x': '대여 대여소명', '보관소(대여소)명_y': '반납 대여소명','위도_x':'대여소 위도', '경도_x':'대여소 경도', '위도_y':'반납소 위도', '경도_y':'반납소 경도'})

top_3_merged_yd

<ipython-input-60-9f553ba07bf3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yd['대여 반납'] = df_yd['대여소 번호'].astype(str) + '-' + df_yd['반납대여소번호'].astype(str)


,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,영등포구,207,272,2352,여의나루역 1번출구 앞,37.527157,126.931900,당산육갑문,37.535339,126.903679,0.510258
1,영등포구,272,207,2088,당산육갑문,37.535339,126.903679,여의나루역 1번출구 앞,37.527157,126.931900,0.452984
2,영등포구,207,249,1911,여의나루역 1번출구 앞,37.527157,126.931900,여의도중학교 옆,37.524349,126.936119,0.414585


## 4. 구로구

In [ ]:
df = pd.read_csv ('/content/구로구_2022_따릉이대여이력.csv', encoding = 'cp949')

# 자치구 컬럼이 '구로구'이고 대여소번호와 반납 대여소번호가 서로 다른 경우 추출
df_gr = df.loc[(df['대여 자치구'] == '구로구') & (df['대여소 번호'] != df['반납대여소번호'])]

# 대여 대여소번호와 반납 대여소번호를 조합하여 새로운 컬럼 생성
df_gr['대여 반납'] = df_gr['대여소 번호'].astype(str) + '-' + df_gr['반납대여소번호'].astype(str)

# 가장 많이 나온 상위 3개의 조합과 그 횟수 추출
top_3 = df_gr['대여 반납'].value_counts().head(3)

# 대여소 번호가 서로 다른 경우를 제외한 데이터만 추출
df_gr_unique = df_gr[df_gr['대여소 번호'] != df_gr['반납대여소번호']]

# 대여 대여소번호와 반납 대여소번호가 서로 다른 조합 상위 3개와 그 횟수 추출
top_3 = df_gr_unique.groupby(['대여 자치구', '대여소 번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(3, columns=['빈도수'])

# 대여소 번호와 대여소 이름 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()


# top_3 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_3_merged_gr = pd.merge(top_3, df_station_name, left_on='대여소 번호', right_on='대여소번호', how='left')
top_3_merged_gr = pd.merge(top_3_merged_gr, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')
# 빈도수의 백분율 계산
top_3_merged_gr['백분율'] = (top_3_merged_gr['빈도수'] / len(df_gr)) * 100

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_3_merged_gr = top_3_merged_gr.drop(columns=['대여소번호_x', '대여소번호_y'])
top_3_merged_gr = top_3_merged_gr.rename(columns={'보관소(대여소)명_x': '대여 대여소명', '보관소(대여소)명_y': '반납 대여소명','위도_x':'대여소 위도', '경도_x':'대여소 경도', '위도_y':'반납소 위도', '경도_y':'반납소 경도'})

top_3_merged_gr

<ipython-input-61-9ef043351bd5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gr['대여 반납'] = df_gr['대여소 번호'].astype(str) + '-' + df_gr['반납대여소번호'].astype(str)


,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,구로구,1911,1906,1248,구로디지털단지역 앞,37.484940,126.901321,신도림역 1번 출구 앞,37.509720,126.890099,0.608267
1,구로구,1906,1911,1175,신도림역 1번 출구 앞,37.509720,126.890099,구로디지털단지역 앞,37.484940,126.901321,0.572687
2,구로구,1925,1991,954,동양미래대학교 정문 옆,37.499889,126.867699,오류동역 맞은편,37.496761,126.844963,0.464973


## 5. 금천구

In [ ]:
df = pd.read_csv ('/content/금천구_2022_따릉이대여이력.csv', encoding = 'cp949')

# 자치구 컬럼이 '금천구'이고 대여소번호와 반납 대여소번호가 서로 다른 경우 추출
df_gc = df.loc[(df['대여 자치구'] == '금천구') & (df['대여소 번호'] != df['반납대여소번호'])]

# 대여 대여소번호와 반납 대여소번호를 조합하여 새로운 컬럼 생성
df_gc['대여 반납'] = df_gc['대여소 번호'].astype(str) + '-' + df_gc['반납대여소번호'].astype(str)

# 가장 많이 나온 상위 3개의 조합과 그 횟수 추출
top_3 = df_gc['대여 반납'].value_counts().head(3)

# 대여소 번호가 서로 다른 경우를 제외한 데이터만 추출
df_gc_unique = df_gc[df_gc['대여소 번호'] != df_gc['반납대여소번호']]

# 대여 대여소번호와 반납 대여소번호가 서로 다른 조합 상위 3개와 그 횟수 추출
top_3 = df_gc_unique.groupby(['대여 자치구', '대여소 번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(3, columns=['빈도수'])

# 대여소 번호와 대여소 이름 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()


# top_3 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_3_merged_gc = pd.merge(top_3, df_station_name, left_on='대여소 번호', right_on='대여소번호', how='left')
top_3_merged_gc = pd.merge(top_3_merged_gc, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')
# 빈도수의 백분율 계산
top_3_merged_gc['백분율'] = (top_3_merged_gc['빈도수'] / len(df_gc)) * 100

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_3_merged_gc = top_3_merged_gc.drop(columns=['대여소번호_x', '대여소번호_y'])
top_3_merged_gc = top_3_merged_gc.rename(columns={'보관소(대여소)명_x': '대여 대여소명', '보관소(대여소)명_y': '반납 대여소명','위도_x':'대여소 위도', '경도_x':'대여소 경도', '위도_y':'반납소 위도', '경도_y':'반납소 경도'})

ㅊ

<ipython-input-62-d491fdde600d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gc['대여 반납'] = df_gc['대여소 번호'].astype(str) + '-' + df_gc['반납대여소번호'].astype(str)


,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,금천구,1867,1846,948,대륭포스트타워6차 앞,37.480888,126.883804,롯데캐슬골드파크1차 동문,37.460171,126.896042,0.951521
1,금천구,3963,1867,827,에이스 하드웨어(시흥대로 396) 앞,37.468658,126.898262,대륭포스트타워6차 앞,37.480888,126.883804,0.830071
2,금천구,1848,1846,774,벽산 디지털밸리 5차,37.476276,126.885391,롯데캐슬골드파크1차 동문,37.460171,126.896042,0.776874


In [ ]:
df_top3_total = pd.concat([top_3_merged, top_3_merged_gr, top_3_merged_yd, top_3_merged_gs, top_3_merged_gc])
df_top3_total

,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,양천구,703,734,1002,오목교역 7번출구 앞,37.523628,126.875244,신트리공원 입구,37.513950,126.856056,0.302263
1,양천구,770,787,928,목동역5번출구 교통정보센터 앞,37.525715,126.864868,양천아파트 입구,37.507904,126.862648,0.279941
2,양천구,787,792,843,양천아파트 입구,37.507904,126.862648,목동트라팰리스 웨스턴에비뉴,37.525372,126.870926,0.254299
0,구로구,1911,1906,1248,구로디지털단지역 앞,37.484940,126.901321,신도림역 1번 출구 앞,37.509720,126.890099,0.608267
1,구로구,1906,1911,1175,신도림역 1번 출구 앞,37.509720,126.890099,구로디지털단지역 앞,37.484940,126.901321,0.572687
2,구로구,1925,1991,954,동양미래대학교 정문 옆,37.499889,126.867699,오류동역 맞은편,37.496761,126.844963,0.464973
0,영등포구,207,272,2352,여의나루역 1번출구 앞,37.527157,126.931900,당산육갑문,37.535339,126.903679,0.510258
1,영등포구,272,207,2088,당산육갑문,37.535339,126.903679,여의나루역 1번출구 앞,37.527157,126.931900,0.452984
2,영등포구,207,249,1911,여의나루역 1번출구 앞,37.527157,126.931900,여의도중학교 옆,37.524349,126.936119,0.414585
0,강서구,1153,1150,1079,"발산역 1번, 9번 인근 대여소",37.558949,126.837174,송정역 1번출구,37.561531,126.810699,0.136903


In [ ]:
df_top3_total.to_csv('구별_top3_total.csv', index=False, encoding='cp949')

In [6]:
# 강북구, 도봉구, 노원구, 중랑구, 강동구 진행

import pandas as pd

df_rental = pd.read_csv('/content/drive/MyDrive/merged_2022_따릉이대여이력_수정.csv', encoding='cp949')
# 대여소 정보 데이터를 읽어옵니다.
df_station = pd.read_csv('/content/drive/MyDrive/대여소정보.csv')

In [8]:
df=df_rental

In [9]:
df_filtered = df[((df['대여 자치구'] == '강북구') & (df['반납 자치구'] == '강북구'))]

df_merged = pd.merge(df_filtered, df_station[['대여소번호', '경도', '위도']], left_on='대여 대여소번호', right_on='대여소번호')
df_merged = pd.merge(df_merged, df_station[['대여소번호', '경도', '위도']], left_on='반납대여소번호', right_on='대여소번호', suffixes=('_대여', '_반납'))
df_merged = df_merged.rename(columns={'대여 대여소번호': '대여소 번호', '대여 대여소명': '대여소명', '위도_대여':'대여소 위도', '경도_대여':'대여소 경도', '반납대여소명':'반납소명','위도_반납':'반납소 위도', '경도_반납':'반납소 경도'})
df_merged.drop(['대여소번호_대여', '대여소번호_반납'], axis=1, inplace=True)

df_merged.to_csv('강북구_2022_따릉이대여이력.csv', index=False, encoding='cp949')

In [10]:
df_filtered = df[((df['대여 자치구'] == '도봉구') & (df['반납 자치구'] == '도봉구'))]

df_merged = pd.merge(df_filtered, df_station[['대여소번호', '경도', '위도']], left_on='대여 대여소번호', right_on='대여소번호')
df_merged = pd.merge(df_merged, df_station[['대여소번호', '경도', '위도']], left_on='반납대여소번호', right_on='대여소번호', suffixes=('_대여', '_반납'))
df_merged = df_merged.rename(columns={'대여 대여소번호': '대여소 번호', '대여 대여소명': '대여소명', '위도_대여':'대여소 위도', '경도_대여':'대여소 경도', '반납대여소명':'반납소명','위도_반납':'반납소 위도', '경도_반납':'반납소 경도'})
df_merged.drop(['대여소번호_대여', '대여소번호_반납'], axis=1, inplace=True)

df_merged.to_csv('도봉구_2022_따릉이대여이력.csv', index=False, encoding='cp949')

In [11]:
df_filtered = df[((df['대여 자치구'] == '노원구') & (df['반납 자치구'] == '노원구'))]

df_merged = pd.merge(df_filtered, df_station[['대여소번호', '경도', '위도']], left_on='대여 대여소번호', right_on='대여소번호')
df_merged = pd.merge(df_merged, df_station[['대여소번호', '경도', '위도']], left_on='반납대여소번호', right_on='대여소번호', suffixes=('_대여', '_반납'))
df_merged = df_merged.rename(columns={'대여 대여소번호': '대여소 번호', '대여 대여소명': '대여소명', '위도_대여':'대여소 위도', '경도_대여':'대여소 경도', '반납대여소명':'반납소명','위도_반납':'반납소 위도', '경도_반납':'반납소 경도'})
df_merged.drop(['대여소번호_대여', '대여소번호_반납'], axis=1, inplace=True)

df_merged.to_csv('노원구_2022_따릉이대여이력.csv', index=False, encoding='cp949')

In [12]:
df_filtered = df[((df['대여 자치구'] == '중랑구') & (df['반납 자치구'] == '중랑구'))]

df_merged = pd.merge(df_filtered, df_station[['대여소번호', '경도', '위도']], left_on='대여 대여소번호', right_on='대여소번호')
df_merged = pd.merge(df_merged, df_station[['대여소번호', '경도', '위도']], left_on='반납대여소번호', right_on='대여소번호', suffixes=('_대여', '_반납'))
df_merged = df_merged.rename(columns={'대여 대여소번호': '대여소 번호', '대여 대여소명': '대여소명', '위도_대여':'대여소 위도', '경도_대여':'대여소 경도', '반납대여소명':'반납소명','위도_반납':'반납소 위도', '경도_반납':'반납소 경도'})
df_merged.drop(['대여소번호_대여', '대여소번호_반납'], axis=1, inplace=True)

df_merged.to_csv('중랑구_2022_따릉이대여이력.csv', index=False, encoding='cp949')

In [13]:
df_filtered = df[((df['대여 자치구'] == '강동구') & (df['반납 자치구'] == '강동구'))]

df_merged = pd.merge(df_filtered, df_station[['대여소번호', '경도', '위도']], left_on='대여 대여소번호', right_on='대여소번호')
df_merged = pd.merge(df_merged, df_station[['대여소번호', '경도', '위도']], left_on='반납대여소번호', right_on='대여소번호', suffixes=('_대여', '_반납'))
df_merged = df_merged.rename(columns={'대여 대여소번호': '대여소 번호', '대여 대여소명': '대여소명', '위도_대여':'대여소 위도', '경도_대여':'대여소 경도', '반납대여소명':'반납소명','위도_반납':'반납소 위도', '경도_반납':'반납소 경도'})
df_merged.drop(['대여소번호_대여', '대여소번호_반납'], axis=1, inplace=True)

df_merged.to_csv('강동구_2022_따릉이대여이력.csv', index=False, encoding='cp949')

## 최다조합 추출

## 1.강북구

In [47]:
# csv 파일을 pandas DataFrame으로 읽어오기
df_gb = pd.read_csv('/content/drive/MyDrive/강북구_2022_따릉이대여이력.csv', encoding='cp949')

# '대여 대여소번호'와 '반납 대여소 번호' 컬럼의 조합 빈도수 계산
combo_counts = df_gb.groupby(['대여소 번호', '반납대여소번호']).size().reset_index(name='count')

# 가장 많이 나온 조합과 횟수 출력
max_combo = combo_counts.loc[combo_counts['count'].idxmax()]
print('가장 많이 이용한 구간:', max_combo['대여소 번호'], max_combo['반납대여소번호'])
print('2022년 이용 횟수:', max_combo['count'])
print('2022년 강북구 내 이동 전체 따릉이 이용 횟수:',  len(df_gb))

가장 많이 이용한 구간: 1512 1512
2022년 이용 횟수: 2819
2022년 강북구 내 이동 전체 따릉이 이용 횟수: 106592


In [48]:
df_gb_max = df_gb[(df_gb['대여 자치구'] == '강북구') & (df_gb['대여소 번호'] == 1512) & (df_gb['반납대여소번호'] == 1512)]

# 결과 출력
df_gb_max.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
37815,2022.12.1 19:02,1512,강북중학교 앞,2022.12.1 19:49,1512,강북중학교 앞,46.0,3180.00,ST-862,ST-862,강북구,강북구,127.026566,37.644737,127.026566,37.644737
37816,2022.12.2 17:06,1512,강북중학교 앞,2022.12.2 18:31,1512,강북중학교 앞,84.0,8280.00,ST-862,ST-862,강북구,강북구,127.026566,37.644737,127.026566,37.644737
37817,2022.12.3 14:40,1512,강북중학교 앞,2022.12.3 15:33,1512,강북중학교 앞,53.0,13133.36,ST-862,ST-862,강북구,강북구,127.026566,37.644737,127.026566,37.644737
37818,2022.12.5 15:47,1512,강북중학교 앞,2022.12.5 17:28,1512,강북중학교 앞,100.0,4719.01,ST-862,ST-862,강북구,강북구,127.026566,37.644737,127.026566,37.644737
37819,2022.12.6 11:39,1512,강북중학교 앞,2022.12.6 12:22,1512,강북중학교 앞,43.0,8230.00,ST-862,ST-862,강북구,강북구,127.026566,37.644737,127.026566,37.644737


### 2. 도봉구

In [49]:
# csv 파일을 pandas DataFrame으로 읽어오기
df_db = pd.read_csv('/content/drive/MyDrive/도봉구_2022_따릉이대여이력.csv', encoding='cp949')
df_db.dropna(inplace=True)

# '대여 대여소번호'와 '반납 대여소 번호' 컬럼의 조합 빈도수 계산
combo_counts = df_db.groupby(['대여소 번호', '반납대여소번호']).size().reset_index(name='count')

# 가장 많이 나온 조합과 횟수 출력
max_combo = combo_counts.loc[combo_counts['count'].idxmax()]
print('가장 많이 이용한 구간:', max_combo['대여소 번호'], max_combo['반납대여소번호'])
print('2022년 이용 횟수:', max_combo['count'])
print('2022년 도봉구 내 이동 전체 따릉이 이용 횟수:',  len(df_db))

가장 많이 이용한 구간: 4076 4076
2022년 이용 횟수: 1984
2022년 도봉구 내 이동 전체 따릉이 이용 횟수: 84491


In [50]:
df_db_max = df_db[(df_db['대여 자치구'] == '도봉구') & (df_db['대여소 번호'] == 4076) & (df_db['반납대여소번호'] == 4076)]

# 결과 출력
df_db_max.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
20041,2022.12.2 9:14,4076,도봉통합복지센터 뒤 중랑천 변,2022.12.2 10:03,4076,도봉통합복지센터 뒤 중랑천 변,49.0,9347.58,ST-2549,ST-2549,도봉구,도봉구,127.051277,37.659996,127.051277,37.659996
20042,2022.12.2 14:41,4076,도봉통합복지센터 뒤 중랑천 변,2022.12.2 16:25,4076,도봉통합복지센터 뒤 중랑천 변,104.0,12920.00,ST-2549,ST-2549,도봉구,도봉구,127.051277,37.659996,127.051277,37.659996
20043,2022.12.2 22:04,4076,도봉통합복지센터 뒤 중랑천 변,2022.12.2 23:45,4076,도봉통합복지센터 뒤 중랑천 변,100.0,20880.00,ST-2549,ST-2549,도봉구,도봉구,127.051277,37.659996,127.051277,37.659996
20044,2022.12.3 12:31,4076,도봉통합복지센터 뒤 중랑천 변,2022.12.3 12:55,4076,도봉통합복지센터 뒤 중랑천 변,24.0,6440.00,ST-2549,ST-2549,도봉구,도봉구,127.051277,37.659996,127.051277,37.659996
20045,2022.12.3 15:30,4076,도봉통합복지센터 뒤 중랑천 변,2022.12.3 16:00,4076,도봉통합복지센터 뒤 중랑천 변,30.0,4200.00,ST-2549,ST-2549,도봉구,도봉구,127.051277,37.659996,127.051277,37.659996


## 3. 노원구

In [51]:
# csv 파일을 pandas DataFrame으로 읽어오기
df_nw = pd.read_csv('/content/drive/MyDrive/노원구_2022_따릉이대여이력.csv', encoding='cp949')
df_nw.dropna(inplace=True)

# '대여 대여소번호'와 '반납 대여소 번호' 컬럼의 조합 빈도수 계산
combo_counts = df_nw.groupby(['대여소 번호', '반납대여소번호']).size().reset_index(name='count')

# 가장 많이 나온 조합과 횟수 출력
max_combo = combo_counts.loc[combo_counts['count'].idxmax()]
print('가장 많이 이용한 구간:', max_combo['대여소 번호'], max_combo['반납대여소번호'])
print('2022년 이용 횟수:', max_combo['count'])
print('2022년 노원구 내 이동 전체 따릉이 이용 횟수:',  len(df_nw))

가장 많이 이용한 구간: 4029 4029
2022년 이용 횟수: 1792
2022년 노원구 내 이동 전체 따릉이 이용 횟수: 291802


In [52]:
df_nw_max = df_nw[(df_nw['대여 자치구'] == '노원구') & (df_nw['대여소 번호'] == 4029) & (df_nw['반납대여소번호'] == 4029)]

# 결과 출력
df_nw_max.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
360173,2022.12.1 5:01,4029,청소년수련관 후문,2022.12.1 5:56,4029,청소년수련관 후문,55.0,5310.00,ST-2718,ST-2718,노원구,노원구,127.060417,37.639664,127.060417,37.639664
360174,2022.12.1 16:19,4029,청소년수련관 후문,2022.12.1 17:55,4029,청소년수련관 후문,96.0,4000.00,ST-2718,ST-2718,노원구,노원구,127.060417,37.639664,127.060417,37.639664
360175,2022.12.1 18:20,4029,청소년수련관 후문,2022.12.1 18:56,4029,청소년수련관 후문,35.0,6970.00,ST-2718,ST-2718,노원구,노원구,127.060417,37.639664,127.060417,37.639664
360176,2022.12.1 19:59,4029,청소년수련관 후문,2022.12.1 21:42,4029,청소년수련관 후문,102.0,3450.00,ST-2718,ST-2718,노원구,노원구,127.060417,37.639664,127.060417,37.639664
360177,2022.12.2 4:57,4029,청소년수련관 후문,2022.12.2 6:10,4029,청소년수련관 후문,72.0,5619.81,ST-2718,ST-2718,노원구,노원구,127.060417,37.639664,127.060417,37.639664


## 4. 중랑구

In [53]:
# csv 파일을 pandas DataFrame으로 읽어오기
df_jr = pd.read_csv('/content/drive/MyDrive/중랑구_2022_따릉이대여이력.csv', encoding='cp949')
df_jr.dropna(inplace=True)

# '대여 대여소번호'와 '반납 대여소 번호' 컬럼의 조합 빈도수 계산
combo_counts = df_jr.groupby(['대여소 번호', '반납대여소번호']).size().reset_index(name='count')

# 가장 많이 나온 조합과 횟수 출력
max_combo = combo_counts.loc[combo_counts['count'].idxmax()]
print('가장 많이 이용한 구간:', max_combo['대여소 번호'], max_combo['반납대여소번호'])
print('2022년 이용 횟수:', max_combo['count'])
print('2022년 중랑구 내 이동 전체 따릉이 이용 횟수:',  len(df_jr))

가장 많이 이용한 구간: 1452 1452
2022년 이용 횟수: 1944
2022년 중랑구 내 이동 전체 따릉이 이용 횟수: 107733


In [54]:

df_jr_max = df_jr[(df_jr['대여 자치구'] == '중랑구') & (df_jr['대여소 번호'] == 1452) & (df_jr['반납대여소번호'] == 1452)]

# 결과 출력
df_jr_max.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
49624,2022.12.1 19:05,1452,겸재교 진입부,2022.12.1 20:02,1452,겸재교 진입부,56.0,2624.21,ST-1335,ST-1335,중랑구,중랑구,127.07505,37.585655,127.07505,37.585655
49625,2022.12.2 10:40,1452,겸재교 진입부,2022.12.2 11:44,1452,겸재교 진입부,64.0,2610.00,ST-1335,ST-1335,중랑구,중랑구,127.07505,37.585655,127.07505,37.585655
49626,2022.12.2 22:12,1452,겸재교 진입부,2022.12.2 22:57,1452,겸재교 진입부,45.0,10500.00,ST-1335,ST-1335,중랑구,중랑구,127.07505,37.585655,127.07505,37.585655
49627,2022.12.3 12:19,1452,겸재교 진입부,2022.12.3 13:08,1452,겸재교 진입부,49.0,7250.00,ST-1335,ST-1335,중랑구,중랑구,127.07505,37.585655,127.07505,37.585655
49628,2022.12.4 18:50,1452,겸재교 진입부,2022.12.4 19:04,1452,겸재교 진입부,13.0,2167.11,ST-1335,ST-1335,중랑구,중랑구,127.07505,37.585655,127.07505,37.585655


## 5. 강동구

In [55]:
# csv 파일을 pandas DataFrame으로 읽어오기
df_gd = pd.read_csv('/content/drive/MyDrive/강동구_2022_따릉이대여이력.csv', encoding='cp949')
df_gd.dropna(inplace=True)

# '대여 대여소번호'와 '반납 대여소 번호' 컬럼의 조합 빈도수 계산
combo_counts = df_gd.groupby(['대여소 번호', '반납대여소번호']).size().reset_index(name='count')

# 가장 많이 나온 조합과 횟수 출력
max_combo = combo_counts.loc[combo_counts['count'].idxmax()]
print('가장 많이 이용한 구간:', max_combo['대여소 번호'], max_combo['반납대여소번호'])
print('2022년 이용 횟수:', max_combo['count'])
print('2022년 강동구 내 이동 전체 따릉이 이용 횟수:',  len(df_gd))

가장 많이 이용한 구간: 1008 1008
2022년 이용 횟수: 1565
2022년 강동구 내 이동 전체 따릉이 이용 횟수: 194855


In [56]:
df_gd_max = df_gd[(df_gd['대여 자치구'] == '강동구') & (df_gd['대여소 번호'] == 1008) & (df_gd['반납대여소번호'] == 1008)]

# 결과 출력
df_gd_max.head()

,대여일시,대여소 번호,대여소명,반납일시,반납대여소번호,반납소명,이용시간(분),이용거리(M),대여대여소ID,반납대여소ID,대여 자치구,반납 자치구,대여소 경도,대여소 위도,반납소 경도,반납소 위도
81634,2022.12.1 11:51,1008,암사역 3번출구(국민은행앞),2022.12.1 12:23,1008,암사역 3번출구(국민은행앞),31.0,3010.00,ST-492,ST-492,강동구,강동구,127.127182,37.549576,127.127182,37.549576
81635,2022.12.1 15:26,1008,암사역 3번출구(국민은행앞),2022.12.1 16:15,1008,암사역 3번출구(국민은행앞),48.0,9872.12,ST-492,ST-492,강동구,강동구,127.127182,37.549576,127.127182,37.549576
81636,2022.12.3 11:12,1008,암사역 3번출구(국민은행앞),2022.12.3 14:16,1008,암사역 3번출구(국민은행앞),184.0,14500.00,ST-492,ST-492,강동구,강동구,127.127182,37.549576,127.127182,37.549576
81637,2022.12.4 21:13,1008,암사역 3번출구(국민은행앞),2022.12.4 21:46,1008,암사역 3번출구(국민은행앞),32.0,3547.50,ST-492,ST-492,강동구,강동구,127.127182,37.549576,127.127182,37.549576
81638,2022.12.4 21:31,1008,암사역 3번출구(국민은행앞),2022.12.4 22:39,1008,암사역 3번출구(국민은행앞),67.0,8802.91,ST-492,ST-492,강동구,강동구,127.127182,37.549576,127.127182,37.549576


# 대여/반납 대여소 다른 경우 중 최다 구간 추출

## 1. 강북구

In [41]:
# 자치구 컬럼이 '강북구'만 있는 데이터셋 중 대여소번호와 반납 대여소번호가 서로 다른 경우 추출
df_gb = df_gb.loc[(df_gb['대여소 번호'] != df_gb['반납대여소번호'])]

# 대여 대여소번호와 반납 대여소번호를 조합하여 새로운 컬럼 생성
df_gb['대여 반납'] = df_gb['대여소 번호'].astype(str) + '-' + df_gb['반납대여소번호'].astype(str)

# 가장 많이 나온 상위 3개의 조합과 그 횟수 추출
top_3 = df_gb['대여 반납'].value_counts().head(3)

# 대여소 번호가 서로 다른 경우를 제외한 데이터만 추출
df_gb_unique = df_gb[df_gb['대여소 번호'] != df_gb['반납대여소번호']]

# 대여 대여소번호와 반납 대여소번호가 서로 다른 조합 상위 3개와 그 횟수 추출
top_3 = df_gb_unique.groupby(['대여 자치구', '대여소 번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(3, columns=['빈도수'])

# 대여소 번호와 대여소 이름 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()


# top_3 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_3_merged_gb = pd.merge(top_3, df_station_name, left_on='대여소 번호', right_on='대여소번호', how='left')
top_3_merged_gb = pd.merge(top_3_merged_gb, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')
# 빈도수의 백분율 계산
top_3_merged_gb['백분율'] = (top_3_merged_gb['빈도수'] / len(df_gb)) * 100

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_3_merged_gb = top_3_merged_gb.drop(columns=['대여소번호_x', '대여소번호_y'])
top_3_merged_gb = top_3_merged_gb.rename(columns={'보관소(대여소)명_x': '대여 대여소명', '보관소(대여소)명_y': '반납 대여소명','위도_x':'대여소 위도', '경도_x':'대여소 경도', '위도_y':'반납소 위도', '경도_y':'반납소 경도'})

top_3_merged_gb

,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,강북구,1531,1569,587,미아사거리 1번 출구,37.613956,127.030251,수유역2번출구,37.637970,127.025902,0.697589
1,강북구,1531,1535,459,미아사거리 1번 출구,37.613956,127.030251,효성인텔리안 앞,37.636291,127.023376,0.545474
2,강북구,1573,1569,415,현대성우아파트 옆,37.616665,127.022110,수유역2번출구,37.637970,127.025902,0.493185


## 2.도봉구

In [42]:
# 자치구 컬럼이 '도봉구'만 있는 데이터셋 중 대여소번호와 반납 대여소번호가 서로 다른 경우 추출
df_db = df_db.loc[(df_db['대여소 번호'] != df_db['반납대여소번호'])]

# 대여 대여소번호와 반납 대여소번호를 조합하여 새로운 컬럼 생성
df_db['대여 반납'] = df_db['대여소 번호'].astype(str) + '-' + df_db['반납대여소번호'].astype(str)

# 가장 많이 나온 상위 3개의 조합과 그 횟수 추출
top_3 = df_db['대여 반납'].value_counts().head(3)

# 대여소 번호가 서로 다른 경우를 제외한 데이터만 추출
df_db_unique = df_db[df_db['대여소 번호'] != df_db['반납대여소번호']]

# 대여 대여소번호와 반납 대여소번호가 서로 다른 조합 상위 3개와 그 횟수 추출
top_3 = df_db_unique.groupby(['대여 자치구', '대여소 번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(3, columns=['빈도수'])

# 대여소 번호와 대여소 이름 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()


# top_3 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_3_merged_db = pd.merge(top_3, df_station_name, left_on='대여소 번호', right_on='대여소번호', how='left')
top_3_merged_db = pd.merge(top_3_merged_db, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')
# 빈도수의 백분율 계산
top_3_merged_db['백분율'] = (top_3_merged_db['빈도수'] / len(df_db)) * 100

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_3_merged_db = top_3_merged_db.drop(columns=['대여소번호_x', '대여소번호_y'])
top_3_merged_db = top_3_merged_db.rename(columns={'보관소(대여소)명_x': '대여 대여소명', '보관소(대여소)명_y': '반납 대여소명','위도_x':'대여소 위도', '경도_x':'대여소 경도', '위도_y':'반납소 위도', '경도_y':'반납소 경도'})

top_3_merged_db

,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,도봉구,4077,4076,813,도봉제1교,37.679848,127.050385,도봉통합복지센터 뒤 중랑천 변,37.659996,127.051277,1.259782
1,도봉구,4076,4077,674,도봉통합복지센터 뒤 중랑천 변,37.659996,127.051277,도봉제1교,37.679848,127.050385,1.044395
2,도봉구,1750,1720,284,창포원 남쪽 입구,37.688267,127.048897,도봉구청 옆(중랑천변),37.668671,127.047981,0.440071


## 3. 노원구

In [43]:
# 자치구 컬럼이 '노원구'만 있는 데이터셋 중 대여소번호와 반납 대여소번호가 서로 다른 경우 추출
df_nw = df_nw.loc[(df_nw['대여소 번호'] != df_nw['반납대여소번호'])]

# 대여 대여소번호와 반납 대여소번호를 조합하여 새로운 컬럼 생성
df_nw['대여 반납'] = df_nw['대여소 번호'].astype(str) + '-' + df_nw['반납대여소번호'].astype(str)

# 가장 많이 나온 상위 3개의 조합과 그 횟수 추출
top_3 = df_nw['대여 반납'].value_counts().head(3)

# 대여소 번호가 서로 다른 경우를 제외한 데이터만 추출
df_nw_unique = df_nw[df_nw['대여소 번호'] != df_nw['반납대여소번호']]

# 대여 대여소번호와 반납 대여소번호가 서로 다른 조합 상위 3개와 그 횟수 추출
top_3 = df_nw_unique.groupby(['대여 자치구', '대여소 번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(3, columns=['빈도수'])

# 대여소 번호와 대여소 이름 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()


# top_3 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_3_merged_nw = pd.merge(top_3, df_station_name, left_on='대여소 번호', right_on='대여소번호', how='left')
top_3_merged_nw = pd.merge(top_3_merged_nw, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')
# 빈도수의 백분율 계산
top_3_merged_nw['백분율'] = (top_3_merged_nw['빈도수'] / len(df_nw)) * 100

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_3_merged_nw = top_3_merged_nw.drop(columns=['대여소번호_x', '대여소번호_y'])
top_3_merged_nw = top_3_merged_nw.rename(columns={'보관소(대여소)명_x': '대여 대여소명', '보관소(대여소)명_y': '반납 대여소명','위도_x':'대여소 위도', '경도_x':'대여소 경도', '위도_y':'반납소 위도', '경도_y':'반납소 경도'})

top_3_merged_nw

,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,노원구,1601,2917,620,석계역 문화광장 내 자전거 보관소,37.614960,127.064697,신창중학교 후문 맞은편 버스정류장,37.628071,127.046921,0.251015
1,노원구,1679,1610,569,삼육대 도서관,37.642830,127.106667,화랑대역 2번출구 앞,37.620369,127.083641,0.230367
2,노원구,2917,1601,518,신창중학교 후문 맞은편 버스정류장,37.628071,127.046921,석계역 문화광장 내 자전거 보관소,37.614960,127.064697,0.209719


## 4. 중랑구

In [44]:
# 자치구 컬럼이 '중랑구'만 있는 데이터셋 중 대여소번호와 반납 대여소번호가 서로 다른 경우 추출
df_jr = df_jr.loc[(df_jr['대여소 번호'] != df_jr['반납대여소번호'])]

# 대여 대여소번호와 반납 대여소번호를 조합하여 새로운 컬럼 생성
df_jr['대여 반납'] = df_jr['대여소 번호'].astype(str) + '-' + df_jr['반납대여소번호'].astype(str)

# 가장 많이 나온 상위 3개의 조합과 그 횟수 추출
top_3 = df_jr['대여 반납'].value_counts().head(3)

# 대여소 번호가 서로 다른 경우를 제외한 데이터만 추출
df_jr_unique = df_jr[df_jr['대여소 번호'] != df_jr['반납대여소번호']]

# 대여 대여소번호와 반납 대여소번호가 서로 다른 조합 상위 3개와 그 횟수 추출
top_3 = df_jr_unique.groupby(['대여 자치구', '대여소 번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(3, columns=['빈도수'])

# 대여소 번호와 대여소 이름 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()


# top_3 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_3_merged_jr = pd.merge(top_3, df_station_name, left_on='대여소 번호', right_on='대여소번호', how='left')
top_3_merged_jr = pd.merge(top_3_merged_jr, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')
# 빈도수의 백분율 계산
top_3_merged_jr['백분율'] = (top_3_merged_jr['빈도수'] / len(df_jr)) * 100

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_3_merged_jr = top_3_merged_jr.drop(columns=['대여소번호_x', '대여소번호_y'])
top_3_merged_jr = top_3_merged_jr.rename(columns={'보관소(대여소)명_x': '대여 대여소명', '보관소(대여소)명_y': '반납 대여소명','위도_x':'대여소 위도', '경도_x':'대여소 경도', '위도_y':'반납소 위도', '경도_y':'반납소 경도'})

top_3_merged_jr

,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,중랑구,1462,1414,177,동부시장 북문 앞,37.593868,127.076523,사가정역 2번출구,37.580990,127.088242,0.202290
1,중랑구,1451,1414,175,중랑세무서,37.592758,127.072670,사가정역 2번출구,37.580990,127.088242,0.200005
2,중랑구,1458,1452,169,상봉터미널2,37.597340,127.093086,겸재교 진입부,37.585655,127.075050,0.193147


## 5. 강동구

In [45]:
# 자치구 컬럼이 '강동구'만 있는 데이터셋 중 대여소번호와 반납 대여소번호가 서로 다른 경우 추출
df_gd = df_gd.loc[(df_gd['대여소 번호'] != df_gd['반납대여소번호'])]

# 대여 대여소번호와 반납 대여소번호를 조합하여 새로운 컬럼 생성
df_gd['대여 반납'] = df_gd['대여소 번호'].astype(str) + '-' + df_gd['반납대여소번호'].astype(str)

# 가장 많이 나온 상위 3개의 조합과 그 횟수 추출
top_3 = df_gd['대여 반납'].value_counts().head(3)

# 대여소 번호가 서로 다른 경우를 제외한 데이터만 추출
df_gd_unique = df_gd[df_gd['대여소 번호'] != df_gd['반납대여소번호']]

# 대여 대여소번호와 반납 대여소번호가 서로 다른 조합 상위 3개와 그 횟수 추출
top_3 = df_gd_unique.groupby(['대여 자치구', '대여소 번호', '반납대여소번호']).size().reset_index(name='빈도수').nlargest(3, columns=['빈도수'])

# 대여소 번호와 대여소 이름 컬럼만 추출하여 새로운 데이터프레임을 만듭니다.
df_station_name = df_station[['대여소번호', '보관소(대여소)명', '위도', '경도']].drop_duplicates()


# top_3 데이터프레임과 merge 함수를 이용하여 대여소 이름을 추가합니다.
top_3_merged_gd = pd.merge(top_3, df_station_name, left_on='대여소 번호', right_on='대여소번호', how='left')
top_3_merged_gd = pd.merge(top_3_merged_gd, df_station_name, left_on='반납대여소번호', right_on='대여소번호', how='left')
# 빈도수의 백분율 계산
top_3_merged_gd['백분율'] = (top_3_merged_gd['빈도수'] / len(df_gd)) * 100

# 불필요한 컬럼과 중복된 대여소 번호 컬럼을 삭제합니다.
top_3_merged_gd = top_3_merged_gd.drop(columns=['대여소번호_x', '대여소번호_y'])
top_3_merged_gd = top_3_merged_gd.rename(columns={'보관소(대여소)명_x': '대여 대여소명', '보관소(대여소)명_y': '반납 대여소명','위도_x':'대여소 위도', '경도_x':'대여소 경도', '위도_y':'반납소 위도', '경도_y':'반납소 경도'})

top_3_merged_gd

,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,강동구,1090,3675,449,상일동역 2번출구 앞,37.556789,127.166145,강동리버스트8단지,37.571804,127.174469,0.281240
1,강동구,1044,1009,378,굽은다리역,37.545406,127.142677,천호역4번출구(현대백화점),37.538666,127.124245,0.236768
2,강동구,1050,1044,355,둔촌역 3번 출입구,37.526783,127.135574,굽은다리역,37.545406,127.142677,0.222361


In [58]:
df_top3_total = pd.concat([top_3_merged_gb, top_3_merged_nw, top_3_merged_db, top_3_merged_jr, top_3_merged_gd])
df_top3_total

,대여 자치구,대여소 번호,반납대여소번호,빈도수,대여 대여소명,대여소 위도,대여소 경도,반납 대여소명,반납소 위도,반납소 경도,백분율
0,강북구,1531,1569,587,미아사거리 1번 출구,37.613956,127.030251,수유역2번출구,37.637970,127.025902,0.697589
1,강북구,1531,1535,459,미아사거리 1번 출구,37.613956,127.030251,효성인텔리안 앞,37.636291,127.023376,0.545474
2,강북구,1573,1569,415,현대성우아파트 옆,37.616665,127.022110,수유역2번출구,37.637970,127.025902,0.493185
0,노원구,1601,2917,620,석계역 문화광장 내 자전거 보관소,37.614960,127.064697,신창중학교 후문 맞은편 버스정류장,37.628071,127.046921,0.251015
1,노원구,1679,1610,569,삼육대 도서관,37.642830,127.106667,화랑대역 2번출구 앞,37.620369,127.083641,0.230367
2,노원구,2917,1601,518,신창중학교 후문 맞은편 버스정류장,37.628071,127.046921,석계역 문화광장 내 자전거 보관소,37.614960,127.064697,0.209719
0,도봉구,4077,4076,813,도봉제1교,37.679848,127.050385,도봉통합복지센터 뒤 중랑천 변,37.659996,127.051277,1.259782
1,도봉구,4076,4077,674,도봉통합복지센터 뒤 중랑천 변,37.659996,127.051277,도봉제1교,37.679848,127.050385,1.044395
2,도봉구,1750,1720,284,창포원 남쪽 입구,37.688267,127.048897,도봉구청 옆(중랑천변),37.668671,127.047981,0.440071
0,중랑구,1462,1414,177,동부시장 북문 앞,37.593868,127.076523,사가정역 2번출구,37.580990,127.088242,0.202290


In [59]:
df_top3_total.to_csv('강북노원도봉중랑강동_구별_top3_total.csv', index=False, encoding='cp949')